# Final Project: Challenge 2
Contributions From: Ryan Cottone

**Difficulty**: <span style="color:green">$\bullet$</span>

Welcome to the 2nd challenge! The following functions have been changed/added:

- **createLogin**
- **verifyLogin**
- **strangeHashFunction**

Your goal is to find *one* valid password for the given username.

### Utility Functions

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
%run client.ipynb
%run server.ipynb
from utils import *

Here's a copy of `strangeHashFunction` from `utils.py`:

In [ ]:
# NEW 
def strangeHashFunction(s): # s is a string or int
    if type(s) == int:
        s = intToText(s)
    
    total = 0 
    
    for letter in s:
        total += textToInt(s[0])
    
    return hex(hash_SHA256(total % 26))

createLogin and verifyLogin are the same, except they use `strangeHashFunction` instead of `hash_SHA256`.

In [ ]:
ellipticCurve = [0, 7]
ellipticCurveModulus = 115792089237316195423570985008687907853269984665640564039457584007908834671663 
generatorPoint = (55066263022277343669578718895168534326250603453777594175500187360389116729240,
 32670510020758816978083085130507043184471273380659243275938904335757337482424)
generatorPointOrder =115792089237316195423570985008687907852837564279074904382605163141518161494337

### Challenge Reminder 
The following functions have been changed/added:

- **createLogin**
- **verifyLogin**
- **strangeHashFunction**

Your goal is to find *one* valid password for the given username (**not necessarily their *unique* password**).

Can you think of why there might be more than one valid password?

Fill out the function `findLogin` that takes in the username that we want to 'hack' and a function `login`. `login(username, password)` takes in a username and password and returns True if the combination is valid, False otherwise.

Potentially helpful list of lowercase letters...

In [ ]:
string.ascii_lowercase

## Fill this in:

In [ ]:
def findLogin(username, login):
    """
    Given a username and a function login(username, password) that returns True or False whether username, password is correct,
    find a correct login for the username. E.g., a password that will successfully log them in.
    """
    
    ...

Here's an example to test with:

In [ ]:
random.seed(0) # DO NOT CHANGE THIS

client = Client()
server = Server()

server.LOGINS = {}

server.acceptConnection(client.RSA_PUBLIC_KEY)

server_PK, signature = server.publishRSAPublicKey()
assert client.verifyServerCertificate(server_PK, signature)

# Once verified, ask for ECDH parameters
ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature = server.generateECDHParameters()
assert client.verifyECDHParameters(ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature)

# Once ECDH parameters are verified, generate our secret + public value
client_ECDH = client.generateSignedECDHMessage()
server_ECDH = server.generateSignedECDHMessage()

# client gets server ecdh 
client.acceptECDHMessage(server_ECDH[0], server_ECDH[1])
# server gets client ecdh
server.acceptECDHMessage(client_ECDH[0], client_ECDH[1])
assert client.SHARED_SECRET_INT == server.SHARED_SECRET_INT

signupRequest = {'data': '{"iv": "K+e4u0ZEsVQmln3IuzCo9g==", "ciphertext": "Fzz05RGWg03MNviiTS3ibpq0EoiJ+j1xfp7/bAgymylzb+7Rng6lPdpJ8NaLdMLcfR/4hiVDPzRzzFKjB6fbxBCIFIWN8rjysiZWk97EIDs="}',
 'hmac': '7f104635b7e3bb01f22fb699f5e3f82e4551722926c8239e5acb86b19a3964bf'}
server.handleRequest(signupRequest)

In [ ]:
def login(username, password):
    loginRequest = client.generateRequest({'type': 'verifyLogin', 'username': username, 'password': password})
    
    try:
        return server.handleRequest(loginRequest)
    except Exception:
        return False

In [ ]:
onePassword = findLogin('admin', login)

print('Valid password found:', onePassword)
assert login('admin', onePassword)

### Submission

Once you've recovered the password, enter it into the Gradescope page [here](https://www.gradescope.com/courses/406561/assignments/2459002) and copy-paste your **findLogin** code in the same assignment. (Unfortunately our usual autograder does not play nice with this style of challenge)